<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Model_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# อ่านข้อมูล Mrt Transaction

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [4]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


# เลือกข้อมูลตามเดือนที่ต้องการ

In [5]:
df.ThaiMonthName.value_counts()

พฤศจิกายน     1801585
กันยายน       1775869
สิงหาคม       1754984
ตุลาคม        1676494
ธันวาคม       1630557
มิถุนายน      1627196
กรกฎาคม       1606519
พฤษภาคม       1520761
เมษายน        1242435
มีนาคม         948261
มกราคม         829946
กุมภาพันธ์     826162
Name: ThaiMonthName, dtype: int64

In [6]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

In [7]:
jan_df = get_data_by_month("มกราคม")
jan_df.drop('TransactionID', axis=1, inplace=True)
jan_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
15610212,2017-12-01,1,2017-12-01,101,21,Card,Adult-A,Adult,Khlong Bang Phai,PPL,Hua Lamphong,BL,20171201,WD,1,Friday,ศุกร์,December,ธันวาคม,12,4,2,2017
15610213,2017-12-01,21,2017-12-01,101,21,Card,Adult-B,Adult,Khlong Bang Phai,PPL,Hua Lamphong,BL,20171201,WD,1,Friday,ศุกร์,December,ธันวาคม,12,4,2,2017
15610214,2017-12-01,24,2017-12-01,101,21,Card,Student-B,Student,Khlong Bang Phai,PPL,Hua Lamphong,BL,20171201,WD,1,Friday,ศุกร์,December,ธันวาคม,12,4,2,2017
15610215,2017-12-01,58,2017-12-01,101,21,Token,Adult,Adult,Khlong Bang Phai,PPL,Hua Lamphong,BL,20171201,WD,1,Friday,ศุกร์,December,ธันวาคม,12,4,2,2017
15610216,2017-12-01,58,2017-12-01,101,21,Token,Adult,Adult,Khlong Bang Phai,PPL,Hua Lamphong,BL,20171201,WD,1,Friday,ศุกร์,December,ธันวาคม,12,4,2,2017


# Count ข้อมูลแต่ละสถานี แยกตามแต่ละวัน

In [8]:
tran_df = jan_df[['TransactionDate','StationName','CardType']].groupby(['TransactionDate','StationName']).count()
tran_df.reset_index(inplace=True)
tran_df = tran_df.rename(columns={'CardType': 'PassengerCount'})
tran_df

,TransactionDate,StationName,PassengerCount
0,2017-12-01,Bang Krasor,238
1,2017-12-01,Bang Phlu,127
2,2017-12-01,Bang Rak Noi Tha It,241
3,2017-12-01,Bang Rak Yai,58
4,2017-12-01,Bang Son,236
...,...,...,...
1049,2017-12-31,Tao Poon,399
1050,2017-12-31,Thailand Cultural Centre,2228
1051,2017-12-31,Wong Sawang,94
1052,2017-12-31,Yaek Nonthaburi 1,122


In [9]:

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [10]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-12-01,70652
1,2017-12-15,70116
2,2017-12-08,67455
3,2017-12-22,67193
4,2017-12-14,64795


# Preparation ข้อมูลประเภทของวัน (วันทำงาน , วันหยุด , วันนักขัตฤกษ์)

In [11]:
def get_day_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output
    # df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    # return df

In [12]:
day_type_list = get_day_type_list(jan_df , transaction_df , "DayType")

['WD', 'WE', 'PH']
[['2017-12-01', 70652, 0, 0], ['2017-12-15', 70116, 0, 0], ['2017-12-08', 67455, 0, 0], ['2017-12-22', 67193, 0, 0], ['2017-12-14', 64795, 0, 0], ['2017-12-20', 64665, 0, 0], ['2017-12-21', 64361, 0, 0], ['2017-12-13', 63727, 0, 0], ['2017-12-07', 63424, 0, 0], ['2017-12-06', 63391, 0, 0], ['2017-12-19', 63369, 0, 0], ['2017-12-12', 62547, 0, 0], ['2017-12-18', 62033, 0, 0], ['2017-12-04', 59821, 0, 0], ['2017-12-26', 59307, 0, 0], ['2017-12-25', 58506, 0, 0], ['2017-12-28', 56044, 0, 0], ['2017-12-29', 54006, 0, 0], ['2017-12-27', 53824, 0, 0], ['2017-12-23', 0, 48811, 0], ['2017-12-16', 0, 46065, 0], ['2017-12-02', 0, 43693, 0], ['2017-12-09', 0, 39925, 0], ['2017-12-17', 0, 37460, 0], ['2017-12-03', 0, 35364, 0], ['2017-12-24', 0, 34797, 0], ['2017-12-30', 0, 32379, 0], ['2017-12-10', 0, 32350, 0], ['2017-12-11', 0, 0, 32027], ['2017-12-31', 0, 29769, 0], ['2017-12-05', 0, 0, 28681]]


In [13]:
# day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','WD'])
# day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["WD"] != 0) * 3 )
# day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
# day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
# day_type_temp.drop('WE', axis=1, inplace=True)
# day_type_temp.drop('WD', axis=1, inplace=True)

ValueError: ignored

In [14]:
day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','PH','WD'])
day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["PH"] != 0) * 2 ) + ( (day_type_temp["WD"] != 0) * 3 )
day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
day_type_temp.loc[day_type_temp['DayType'] == 2, 'DayType'] = "PH"
day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
day_type_temp.drop('WE', axis=1, inplace=True)
day_type_temp.drop('PH', axis=1, inplace=True)
day_type_temp.drop('WD', axis=1, inplace=True)

In [15]:
day_type_temp.head()

,Date,DayType
0,2017-12-01,WE
1,2017-12-15,WE
2,2017-12-08,WE
3,2017-12-22,WE
4,2017-12-14,WE


In [16]:
jan_transaction_df = pd.merge(left=tran_df, right=day_type_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType
0,2017-12-01,Bang Krasor,238,WE
1,2017-12-01,Bang Phlu,127,WE
2,2017-12-01,Bang Rak Noi Tha It,241,WE
3,2017-12-01,Bang Rak Yai,58,WE
4,2017-12-01,Bang Son,236,WE
...,...,...,...,...
1049,2017-12-31,Tao Poon,399,PH
1050,2017-12-31,Thailand Cultural Centre,2228,PH
1051,2017-12-31,Wong Sawang,94,PH
1052,2017-12-31,Yaek Nonthaburi 1,122,PH


# Preparation ข้อมูลวันอะไรของสัปดาห์

In [17]:
def get_day_of_week_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output

In [18]:
day_of_week_list = get_day_of_week_list(jan_df , transaction_df , "EnglishDayNameOfWeek")

['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
[['2017-12-01', 70652, 0, 0, 0, 0, 0, 0], ['2017-12-15', 70116, 0, 0, 0, 0, 0, 0], ['2017-12-08', 67455, 0, 0, 0, 0, 0, 0], ['2017-12-22', 67193, 0, 0, 0, 0, 0, 0], ['2017-12-14', 0, 0, 0, 0, 0, 0, 64795], ['2017-12-20', 0, 0, 0, 0, 0, 64665, 0], ['2017-12-21', 0, 0, 0, 0, 0, 0, 64361], ['2017-12-13', 0, 0, 0, 0, 0, 63727, 0], ['2017-12-07', 0, 0, 0, 0, 0, 0, 63424], ['2017-12-06', 0, 0, 0, 0, 0, 63391, 0], ['2017-12-19', 0, 0, 0, 0, 63369, 0, 0], ['2017-12-12', 0, 0, 0, 0, 62547, 0, 0], ['2017-12-18', 0, 0, 0, 62033, 0, 0, 0], ['2017-12-04', 0, 0, 0, 59821, 0, 0, 0], ['2017-12-26', 0, 0, 0, 0, 59307, 0, 0], ['2017-12-25', 0, 0, 0, 58506, 0, 0, 0], ['2017-12-28', 0, 0, 0, 0, 0, 0, 56044], ['2017-12-29', 54006, 0, 0, 0, 0, 0, 0], ['2017-12-27', 0, 0, 0, 0, 0, 53824, 0], ['2017-12-23', 0, 48811, 0, 0, 0, 0, 0], ['2017-12-16', 0, 46065, 0, 0, 0, 0, 0], ['2017-12-02', 0, 43693, 0, 0, 0, 0, 0], ['2017-12-09', 0, 

In [19]:
day_of_week_unique = jan_df['EnglishDayNameOfWeek'].unique().tolist()
day_of_week_unique.insert(0,"Date")

In [20]:
day_of_week_temp = pd.DataFrame(day_of_week_list, columns =day_of_week_unique)
day_of_week_temp['DateOfWeek'] = ( (day_of_week_temp["Sunday"] != 0) * 1 ) + ( (day_of_week_temp["Monday"] != 0) * 2 ) + ( (day_of_week_temp["Tuesday"] != 0) * 3 )+ ( (day_of_week_temp["Wednesday"] != 0) * 4 )+ ( (day_of_week_temp["Thursday"] != 0) * 5 )+ ( (day_of_week_temp["Friday"] != 0) * 6 )+ ( (day_of_week_temp["Saturday"] != 0) * 7 )
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 1, 'DateOfWeek'] = "Sunday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 2, 'DateOfWeek'] = "Monday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 3, 'DateOfWeek'] = "Tuesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 4, 'DateOfWeek'] = "Wednesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 5, 'DateOfWeek'] = "Thursday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 6, 'DateOfWeek'] = "Friday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 7, 'DateOfWeek'] = "Saturday"
for col in jan_df['EnglishDayNameOfWeek'].unique().tolist():
  day_of_week_temp.drop(col, axis=1, inplace=True)

In [21]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=day_of_week_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek
0,2017-12-01,Bang Krasor,238,WE,Friday
1,2017-12-01,Bang Phlu,127,WE,Friday
2,2017-12-01,Bang Rak Noi Tha It,241,WE,Friday
3,2017-12-01,Bang Rak Yai,58,WE,Friday
4,2017-12-01,Bang Son,236,WE,Friday
...,...,...,...,...,...
1049,2017-12-31,Tao Poon,399,PH,Sunday
1050,2017-12-31,Thailand Cultural Centre,2228,PH,Sunday
1051,2017-12-31,Wong Sawang,94,PH,Sunday
1052,2017-12-31,Yaek Nonthaburi 1,122,PH,Sunday


# Preparation ข้อมูล วันที่เท่าไหร่ของเดือน และ เป็นช่วงไหนของเดือน (ต้นเดือน , กลางเดือน , สิ้นเดือน)

In [22]:
df_day_number_of_month = pd.DataFrame(columns=['DayNumberOfMonth','MonthPeriod','MonthNumberOfYear','CalendarQuater'])
df_day_number_of_month["DayNumberOfMonth"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthPeriod"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthNumberOfYear"] =  jan_transaction_df['TransactionDate'].str.slice(5, 7).astype('int')

In [23]:
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') <= 5, 'MonthPeriod'] = 1
df_day_number_of_month.loc[( (df_day_number_of_month['MonthPeriod'].astype('int') > 5) & (df_day_number_of_month['MonthPeriod'].astype('int') < 25) ), 'MonthPeriod'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') >= 25, 'MonthPeriod'] = 3

df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 1, 'MonthPeriod'] = "Early"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 2, 'MonthPeriod'] = "Mid"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 3, 'MonthPeriod'] = "Late"

In [24]:
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 1, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 2, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 3, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 4, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 5, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 6, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 7, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 8, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 9, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 10, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 11, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 12, 'CalendarQuater'] = 4

In [25]:
jan_transaction_df = jan_transaction_df.join(df_day_number_of_month)
jan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater
0,2017-12-01,Bang Krasor,238,WE,Friday,1,Early,12,4
1,2017-12-01,Bang Phlu,127,WE,Friday,1,Early,12,4
2,2017-12-01,Bang Rak Noi Tha It,241,WE,Friday,1,Early,12,4
3,2017-12-01,Bang Rak Yai,58,WE,Friday,1,Early,12,4
4,2017-12-01,Bang Son,236,WE,Friday,1,Early,12,4
...,...,...,...,...,...,...,...,...,...
1049,2017-12-31,Tao Poon,399,PH,Sunday,31,Late,12,4
1050,2017-12-31,Thailand Cultural Centre,2228,PH,Sunday,31,Late,12,4
1051,2017-12-31,Wong Sawang,94,PH,Sunday,31,Late,12,4
1052,2017-12-31,Yaek Nonthaburi 1,122,PH,Sunday,31,Late,12,4


# อ่านข้อมูล Events

In [26]:
events_df = pd.read_csv("gdrive/My Drive/NIDA/Events.csv")
events_df.drop('EventsId', axis=1, inplace=True)

In [27]:
events_df.head()

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22


In [28]:
events_df.shape

(920, 6)

In [29]:
events_df.MRTStation.unique()

array(['Queen Sirikit National Convention Centre', 'Phahon Yothin',
       'Phra Ram 9', 'Sukhumvit'], dtype=object)

In [30]:
jan_transaction_df.StationName.unique()

array(['Bang Krasor', 'Bang Phlu', 'Bang Rak Noi Tha It', 'Bang Rak Yai',
       'Bang Son', 'Bang Sue', 'Chatuchak Park', 'Hua Lamphong',
       'Huai Khwang', 'Kamphaeng Phet', 'Khlong Bang Phai', 'Khlong Toei',
       'Lat Phrao', 'Lumphini', 'Ministry of Public Health',
       'Nonthaburi Civic Center', 'Phahon Yothin', 'Phetchaburi',
       'Phra Nang Klao Bridge', 'Phra Ram 9',
       'Queen Sirikit National Convention Centre', 'Ratchadaphisek',
       'Sai Ma', 'Sam Yaek Bang Yai', 'Sam Yan', 'Si Lom', 'Sukhumvit',
       'Sutthisan', 'Talad Bang Yai', 'Tao Poon',
       'Thailand Cultural Centre', 'Wong Sawang', 'Yaek Nonthaburi 1',
       'Yaek Tiwanon'], dtype=object)

In [31]:
events_df.loc[events_df.EventsDate == "2/29/2017" , "EventsDate"] = "2017-02-29"

In [32]:
events_df["EventsGroup"] = np.nan
events_df.loc[events_df.EventsType.isin(["ทั่วไป"]) , "EventsGroup"] = "General"
events_df.loc[events_df.EventsType.isin(["แฟชั่น","ความบันเทิง","Design"]) , "EventsGroup"] = "Fasion"
events_df.loc[events_df.EventsType.isin(["ที่อยู่อาศัย"]) , "EventsGroup"] = "Residence"
events_df.loc[events_df.EventsType.isin(["อาหาร"]) , "EventsGroup"] = "Foods"
events_df.loc[events_df.EventsType.isin(["กีฬา"]) , "EventsGroup"] = "Sports"
events_df.loc[events_df.EventsType.isin(["สัมมนา"]) , "EventsGroup"] = "Seminar"
events_df.loc[events_df.EventsType.isin(["สินค้า IT","เครื่องใช้ไฟฟ้า"]) , "EventsGroup"] = "IT"
events_df.loc[events_df.EventsType.isin(["งานแต่ง","ครอบครัว"]) , "EventsGroup"] = "Family"
events_df.loc[events_df.EventsType.isin(["สุขภาพ","Beauty"]) , "EventsGroup"] = "Health"
events_df.loc[events_df.EventsType.isin(["การศึกษา","กฏหมาย"]) , "EventsGroup"] = "Education"
events_df.loc[events_df.EventsType.isin(["ธุรกิจ","อาชีพ","การเงิน"]) , "EventsGroup"] = "Business"
events_df.loc[events_df.EventsType.isin(["สัตว์เลี้ยง","เกษตร"]) , "EventsGroup"] = "Agriculture"
events_df.loc[events_df.EventsType.isin(["หนังสือ"]) , "EventsGroup"] = "Books"
events_df.loc[events_df.EventsType.isin(["ท่องเที่ยว"]) , "EventsGroup"] = "Travel"
events_df.loc[events_df.EventsType.isin(["พาหนะ"]) , "EventsGroup"] = "Vehnicle"
events_df.EventsGroup.value_counts()

General        198
Fasion         171
Residence      100
Foods           71
Sports          67
IT              60
Business        50
Seminar         43
Travel          37
Agriculture     35
Books           28
Family          20
Education       14
Vehnicle        13
Health          13
Name: EventsGroup, dtype: int64

In [33]:
events_df

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate,EventsGroup
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08,Education
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19,Seminar
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20,Seminar
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21,Education
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22,Education
...,...,...,...,...,...,...,...
915,Shine Flea market ตอน Thai Street Food,TERMINAL 21,ทั่วไป,3542,Sukhumvit,2017-09-23,General
916,Shine Flea market ตอน Thai Street Food,TERMINAL 21,ทั่วไป,3542,Sukhumvit,2017-09-24,General
917,Gift Fest 2018,TERMINAL 21,ทั่วไป,1437,Sukhumvit,2017-12-22,General
918,Gift Fest 2018,TERMINAL 21,ทั่วไป,1437,Sukhumvit,2017-12-23,General


In [34]:
events_group_df = pd.get_dummies(data=events_df , columns=["EventsGroup"])
events_group_df.head()

,EvenstName,EventsLocation,EventsType,Interest,MRTStation,EventsDate,EventsGroup_Agriculture,EventsGroup_Books,EventsGroup_Business,EventsGroup_Education,EventsGroup_Family,EventsGroup_Fasion,EventsGroup_Foods,EventsGroup_General,EventsGroup_Health,EventsGroup_IT,EventsGroup_Residence,EventsGroup_Seminar,EventsGroup_Sports,EventsGroup_Travel,EventsGroup_Vehnicle
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2241,Queen Sirikit National Convention Centre,2017-01-08,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-19,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,1565,Queen Sirikit National Convention Centre,2017-01-20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-21,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,1229,Queen Sirikit National Convention Centre,2017-01-22,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [35]:
events_group_df.shape

(920, 21)

In [36]:
events_group_name = ["General","Fasion","Residence","Foods","Sports","Seminar","IT","Family","Health","Education","Business","Agriculture","Books","Travel","Vehnicle"]

In [37]:
interest_df = events_df.groupby(["EventsDate","MRTStation","EventsGroup"]).sum()
interest_df

Interest
EventsDate MRTStation    EventsGroup          
2017-01-05 Phahon Yothin Foods            5294
                         General          1553
2017-01-06 Phahon Yothin Foods            5294
                         General          1553
2017-01-07 Phahon Yothin Foods            5294
...                                        ...
2017-12-30 Phahon Yothin General          7341
2017-12-31 Phahon Yothin General          7341
2561-01-01 Phahon Yothin General          7341
2561-01-02 Phahon Yothin General          7341
2561-01-03 Phahon Yothin General          7341

[807 rows x 1 columns]

In [38]:
events_interest_df = pd.pivot_table(data=interest_df , values="Interest" , columns="EventsGroup", index=["EventsDate","MRTStation"])
events_interest_df.reset_index(inplace=True)

In [39]:
events_interest_df

EventsGroup,EventsDate,MRTStation,Agriculture,Books,Business,Education,Family,Fasion,Foods,General,Health,IT,Residence,Seminar,Sports,Travel,Vehnicle
0,2017-01-05,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-06,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-07,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-08,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,5294.0,1553.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-08,Queen Sirikit National Convention Centre,NaN,NaN,NaN,2241.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2017-12-30,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,2017-12-31,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,2561-01-01,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,2561-01-02,Phahon Yothin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
events_interest_df["EventsGroup_Agriculture"] = np.nan
events_interest_df.loc[events_interest_df.Agriculture > 0 , "EventsGroup_Agriculture"] = 1

events_interest_df["EventsGroup_Books"] = np.nan
events_interest_df.loc[events_interest_df.Books > 0 , "EventsGroup_Books"] = 1

events_interest_df["EventsGroup_Business"] = np.nan
events_interest_df.loc[events_interest_df.Business > 0 , "EventsGroup_Business"] = 1

events_interest_df["EventsGroup_Education"] = np.nan
events_interest_df.loc[events_interest_df.Education > 0 , "EventsGroup_Education"] = 1

events_interest_df["EventsGroup_Family"] = np.nan
events_interest_df.loc[events_interest_df.Family > 0 , "EventsGroup_Family"] = 1

events_interest_df["EventsGroup_Fasion"] = np.nan
events_interest_df.loc[events_interest_df.Fasion > 0 , "EventsGroup_Fasion"] = 1

events_interest_df["EventsGroup_Foods"] = np.nan
events_interest_df.loc[events_interest_df.Foods > 0 , "EventsGroup_Foods"] = 1

events_interest_df["EventsGroup_General"] = np.nan
events_interest_df.loc[events_interest_df.General > 0 , "EventsGroup_General"] = 1

events_interest_df["EventsGroup_Health"] = np.nan
events_interest_df.loc[events_interest_df.Health > 0 , "EventsGroup_Health"] = 1

events_interest_df["EventsGroup_IT"] = np.nan
events_interest_df.loc[events_interest_df.IT > 0 , "EventsGroup_IT"] = 1

events_interest_df["EventsGroup_Residence"] = np.nan
events_interest_df.loc[events_interest_df.Residence > 0 , "EventsGroup_Residence"] = 1

events_interest_df["EventsGroup_Seminar"] = np.nan
events_interest_df.loc[events_interest_df.Seminar > 0 , "EventsGroup_Seminar"] = 1

events_interest_df["EventsGroup_Sports"] = np.nan
events_interest_df.loc[events_interest_df.Sports > 0 , "EventsGroup_Sports"] = 1

events_interest_df["EventsGroup_Travel"] = np.nan
events_interest_df.loc[events_interest_df.Travel > 0 , "EventsGroup_Travel"] = 1

events_interest_df["EventsGroup_Vehnicle"] = np.nan
events_interest_df.loc[events_interest_df.Vehnicle > 0 , "EventsGroup_Vehnicle"] = 1

In [41]:
events_interest_df.fillna(0, inplace=True)
events_interest_df.rename(columns={'Agriculture': 'Interest_Agriculture'
                                    , 'Books': 'Interest_Books'
                                    , 'Business': 'Interest_Business'
                                    , 'Education': 'Interest_Education'
                                    , 'Family': 'Interest_Family'
                                    , 'Fasion': 'Interest_Fasion'
                                    , 'Foods': 'Interest_Foods'
                                    , 'General': 'Interest_General'
                                    , 'Health': 'Interest_Health'
                                    , 'IT': 'Interest_IT'
                                    , 'Residence': 'Interest_Residence'
                                    , 'Seminar': 'Interest_Seminar'
                                    , 'Sports': 'Interest_Sports'
                                    , 'Travel': 'Interest_Travel'
                                    , 'Vehnicle': 'Interest_Vehnicle'
                                  }, inplace=True)

events_interest_df['Interest_Total'] = events_interest_df['Interest_Agriculture'] + events_interest_df['Interest_Books'] + events_interest_df['Interest_Business'] + events_interest_df['Interest_Education'] + events_interest_df['Interest_Family'] + events_interest_df['Interest_Fasion'] + events_interest_df['Interest_Foods'] + events_interest_df['Interest_General'] + events_interest_df['Interest_Health'] + events_interest_df['Interest_IT'] + events_interest_df['Interest_Residence'] + events_interest_df['Interest_Seminar'] + events_interest_df['Interest_Sports'] + events_interest_df['Interest_Travel'] + events_interest_df['Interest_Vehnicle']
events_interest_df['Events_Total'] = events_interest_df['EventsGroup_Agriculture'] + events_interest_df['EventsGroup_Books'] + events_interest_df['EventsGroup_Business'] + events_interest_df['EventsGroup_Education'] + events_interest_df['EventsGroup_Family'] + events_interest_df['EventsGroup_Fasion'] + events_interest_df['EventsGroup_Foods'] + events_interest_df['EventsGroup_General'] + events_interest_df['EventsGroup_Health'] + events_interest_df['EventsGroup_IT'] + events_interest_df['EventsGroup_Residence'] + events_interest_df['EventsGroup_Seminar'] + events_interest_df['EventsGroup_Sports'] + events_interest_df['EventsGroup_Travel'] + events_interest_df['EventsGroup_Vehnicle']
events_interest_df

EventsGroup,EventsDate,MRTStation,Interest_Agriculture,Interest_Books,Interest_Business,Interest_Education,Interest_Family,Interest_Fasion,Interest_Foods,Interest_General,Interest_Health,Interest_IT,Interest_Residence,Interest_Seminar,Interest_Sports,Interest_Travel,Interest_Vehnicle,EventsGroup_Agriculture,EventsGroup_Books,EventsGroup_Business,EventsGroup_Education,EventsGroup_Family,EventsGroup_Fasion,EventsGroup_Foods,EventsGroup_General,EventsGroup_Health,EventsGroup_IT,EventsGroup_Residence,EventsGroup_Seminar,EventsGroup_Sports,EventsGroup_Travel,EventsGroup_Vehnicle,Interest_Total,Events_Total
0,2017-01-05,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,5294.0,1553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6847.0,2.0
1,2017-01-06,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,5294.0,1553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6847.0,2.0
2,2017-01-07,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,5294.0,1553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6847.0,2.0
3,2017-01-08,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,5294.0,1553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6847.0,2.0
4,2017-01-08,Queen Sirikit National Convention Centre,0.0,0.0,0.0,2241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2241.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2017-12-30,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,1.0
599,2017-12-31,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,1.0
600,2561-01-01,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,1.0
601,2561-01-02,Phahon Yothin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7341.0,1.0


# Preparation ข้อมูล Events

In [42]:
xjan_transaction_df = pd.merge(left=jan_transaction_df,how='left', right=events_interest_df, left_on=['StationName','TransactionDate'], right_on=['MRTStation','EventsDate'])
xjan_transaction_df.drop('EventsDate', axis=1, inplace=True)
xjan_transaction_df.drop('MRTStation', axis=1, inplace=True)
xjan_transaction_df.fillna(0, inplace=True)
xjan_transaction_df

,TransactionDate,StationName,PassengerCount,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater,Interest_Agriculture,Interest_Books,Interest_Business,Interest_Education,Interest_Family,Interest_Fasion,Interest_Foods,Interest_General,Interest_Health,Interest_IT,Interest_Residence,Interest_Seminar,Interest_Sports,Interest_Travel,Interest_Vehnicle,EventsGroup_Agriculture,EventsGroup_Books,EventsGroup_Business,EventsGroup_Education,EventsGroup_Family,EventsGroup_Fasion,EventsGroup_Foods,EventsGroup_General,EventsGroup_Health,EventsGroup_IT,EventsGroup_Residence,EventsGroup_Seminar,EventsGroup_Sports,EventsGroup_Travel,EventsGroup_Vehnicle,Interest_Total,Events_Total
0,2017-12-01,Bang Krasor,238,WE,Friday,1,Early,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-12-01,Bang Phlu,127,WE,Friday,1,Early,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-12-01,Bang Rak Noi Tha It,241,WE,Friday,1,Early,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-12-01,Bang Rak Yai,58,WE,Friday,1,Early,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-12-01,Bang Son,236,WE,Friday,1,Early,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,2017-12-31,Tao Poon,399,PH,Sunday,31,Late,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1050,2017-12-31,Thailand Cultural Centre,2228,PH,Sunday,31,Late,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1051,2017-12-31,Wong Sawang,94,PH,Sunday,31,Late,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1052,2017-12-31,Yaek Nonthaburi 1,122,PH,Sunday,31,Late,12,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
xjan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)